In [1]:
import pandas as pd
import numpy as np
import lightgbm

In [2]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/My Drive/Team 3 Stats 170A/

/content/drive/My Drive/Team 3 Stats 170A


# Get and Sample Data

In [ ]:
!ls subreddit_data/posts

In [ ]:
posts = pd.concat([
    pd.read_csv("./subreddit_data/posts/uci_posts_2010_2023.csv"),
    pd.read_csv("./subreddit_data/posts/ucsc_posts_2010_2023.csv"),
    pd.read_csv("./subreddit_data/posts/ucr_posts_2010_2023.csv"),
    pd.read_csv("./subreddit_data/posts/ucla_posts_2010_2023.csv"),
    pd.read_csv("./subreddit_data/posts/ucmerced_posts_2010_2023.csv"),
    pd.read_csv("./subreddit_data/posts/ucsd_posts_2010_2023.csv"),
    pd.read_csv("./subreddit_data/posts/berkeley_posts_2010_2023.csv"),
    pd.read_csv("./subreddit_data/posts/ucsantabarbara_posts_2010_2023.csv"),
    pd.read_csv("./subreddit_data/posts/ucdavis_posts_2010_2023.csv")
])

ParserError: ignored

In [ ]:
all_posts = posts[['id', 'subreddit', 'title', 'selftext', 'created_utc']].copy()
all_posts['text'] = all_posts['title']
all_posts['text'] = all_posts['text'] + all_posts['selftext'].fillna('NaN')

In [ ]:
all_posts.to_csv('./subreddit_data/posts/all_posts_2010_2023.csv')

In [ ]:
subreddit_data = posts.sample(25000, random_state=30)
posts = None

In [ ]:
subreddit_data = subreddit_data[['id', 'title', 'selftext']]

In [ ]:
subreddit_data['text'] = subreddit_data['title']
subreddit_data['text'] = subreddit_data['text'] + subreddit_data['selftext'].fillna('NaN')

# Get All Hidden States (really slow)

In [5]:
!pip install transformers datasets evaluate concrete-ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.5/168.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# distil-model is a distiled (smaller but similar performance) version of the original model 
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

In [7]:
# Load the pre-trained model
transformer_model = AutoModelForSequenceClassification.from_pretrained(
   "distilroberta-base"
)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 

In [8]:
import torch
import numpy as np
import tqdm

def text_to_tensor(
    list_text_X_train: list,
    transformer_model: AutoModelForSequenceClassification,
    tokenizer: AutoTokenizer,
    device: str,
    batch_size: int = 16,
) -> np.ndarray:
    # Send the model to the device
    transformer_model = transformer_model.to(device)
    output_hidden_states_list = []

    num_batches = len(list_text_X_train) // batch_size + int(len(list_text_X_train) % batch_size != 0)

    for i in tqdm.tqdm(range(num_batches)):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size

        # Tokenize and pad the text in the current batch
        tokenized_text_X_batch = tokenizer(
            list_text_X_train[start_idx:end_idx],
            return_tensors="pt",
            truncation=True,
            padding='longest',
        )

        # Pass the tokens through the transformer model and get the hidden states
        # Only keep the last hidden layer state for now
        tokenized_x = tokenized_text_X_batch['input_ids'].to(device)
        attention_mask = tokenized_text_X_batch['attention_mask'].to(device)
        output_hidden_states = transformer_model(tokenized_x, attention_mask=attention_mask, output_hidden_states=True)[
            1
        ][-1]

        # Average over the tokens axis to get a representation at the text level.
        output_hidden_states = output_hidden_states.mean(dim=1)
        output_hidden_states = output_hidden_states.detach().cpu().numpy()
        output_hidden_states_list.append(output_hidden_states)

    return np.concatenate(output_hidden_states_list, axis=0)


In [9]:
all_posts = pd.read_csv('./subreddit_data/all_posts_2010_2023.csv')

In [10]:
torch.cuda.empty_cache()

In [ ]:
all_posts = all_posts.dropna()
all_hidden_states = text_to_tensor(all_posts['text'].tolist(), transformer_model, tokenizer, device, 32)

  0%|          | 59/18767 [29:48<151:39:43, 29.18s/it]

In [ ]:
np.save('all_hidden_states.npy', all_hidden_states)

# Get Model Scores

In [ ]:
all_hidden_states = np.load('./notebooks/all_hidden_states.npy')

In [ ]:
import lightgbm
academic_model = lightgbm.Booster(model_file='./notebooks/lightgbm_academic_detector.txt')
housing_model = lightgbm.Booster(model_file='./notebooks/lightgbm_housing_detector.txt')

In [ ]:
subreddit_data['academic_score'] = academic_model.predict(all_hidden_states)
subreddit_data['housing_score'] = housing_model.predict(all_hidden_states)

In [ ]:
subreddit_data.to_csv("./notebooks/sample_housing_academic_scored_uci_post_2010_2023_for_labeling.csv", index=False)

# Evaluate Models

# Split into Academic, Housing, and Neither

In [ ]:
neither = subreddit_data.query("academic_score < 0.5 and housing_score < 0.5")
neither.to_csv("./labeling_data/neither_housing_or_academic.csv", index=False)

In [ ]:
housing_and_academic = subreddit_data.query("academic_score > 0.5 and housing_score > 0.5")
housing_and_academic.to_csv("./labeling_data/housing_and_academic.csv", index=False)

In [ ]:
confident_academic = subreddit_data.query("academic_score > 0.8 and housing_score < 0.5")
confident_academic.to_csv("./labeling_data/academic_above_80.csv", index=False)

In [ ]:
not_conf_academic = subreddit_data.query("academic_score <= 0.8 and academic_score > 0.5 and housing_score < 0.5")
not_conf_academic.to_csv("./labeling_data/academic_below_80.csv", index=False)

In [ ]:
confident_housing = subreddit_data.query("academic_score < 0.5 and housing_score > 0.8")
confident_housing.to_csv("./labeling_data/housing_above_80.csv", index=False)

In [ ]:
not_conf_housing = subreddit_data.query("academic_score < 0.5 and housing_score > 0.5 and housing_score <= .8")
not_conf_housing.to_csv("./labeling_data/housing_below_80.csv", index=False)

# Get Samples For Labeling

In [ ]:
# neither samples
neithers_for_labeling = neither[['title', 'selftext', 'id']].sample(2000, random_state=42)

In [ ]:
neithers_for_labeling.columns = ['title', 'text', 'id']

In [ ]:
neithers_for_labeling.to_csv("./labeling_data/Neither_For_Labeling.csv", index=False)

In [ ]:
# academic and housing samples
a_h_for_labeling = pd.concat([
    housing_and_academic[['title', 'selftext', 'id']],
    confident_academic[['title', 'selftext', 'id']].sample(200, random_state=42, replace = True),
    not_conf_academic[['title', 'selftext', 'id']].sample(400, random_state=42, replace = True),
    confident_housing[['title', 'selftext', 'id']].sample(150, random_state=42, replace = True),
    not_conf_housing[['title', 'selftext', 'id']].sample(300, random_state=42, replace = True),
])

In [ ]:
a_h_for_labeling.columns = ['title', 'text', 'id']

In [ ]:
a_h_for_labeling.to_csv("./labeling_data/Housing_Academic_For_Labeling.csv", index=False)